# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [271]:
import pandas as pd
import re

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(0)

Series([], dtype: object)

In [311]:
def year_digits(date):
    new_date = re.search(
        r"(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}\b)", date
    )
    if new_date:
        return (
            new_date.group("month")
            + "-"
            + new_date.group("day")
            + "-"
            + "19"
            + new_date.group("year")
        )
    return date


def date_sorter():
    order = None
    # date_patterns = date_pattern = (
    #     r"(\b0?\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b|\b\d{4}-\d{2}-\d{2}\b|\b\d{1,2}(?:th|st|nd|rd)?\s+\w+\s+\d{4}\b|\b(?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct\\Nov\Dec),?\s+\d{4}\b|\b\.?\w+\.?\s+\d+,?\s+\d{2,4}|\b(?:January|February|March|April|May|June|July|August|September|October|November|December),?\s+\d{2,4}|\d{1,2}/\d{2,4}|19\d\d|20\d\d)"
    # )
    date_patterns = r"(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[\s-]?\d{1,2}?[,\s-]?[\s]?\d{4}|\d{1,2}[/-]\d{4}|\d{4})"
    dates = df.str.extract(date_patterns)
    dates = dates[0].apply(year_digits)
    dates["Date"] = pd.to_datetime(dates[0], errors="coerce", yearfirst=True)

    return dates

    return order  # Your answer here

In [312]:
date_sorter().head(50)

0     03-25-1993
1      6-18-1985
2       7-8-1971
3      9-27-1975
4       2-6-1996
5      7-06-1979
6      5-18-1978
7     10-24-1989
8       3-7-1986
9      4-10-1971
10     5-11-1985
11     4-09-1975
12     8-01-1998
13     1-26-1972
14     5/24/1990
15     1/25/2011
16     4-12-1982
17    10/13/1976
18     4-24-1998
19     5-21-1977
20     7-21-1998
21    10-21-1979
22     3-03-1990
23     2-11-1976
24    07/25/1984
25     4-13-1982
26     9-22-1989
27     9-02-1976
28     9-12-1971
29    10-24-1986
30    03/31/1985
31     7-20-1972
32     4-12-1987
33    06-20-1991
34     5/12/2012
35     3-15-1983
36     2-14-1973
37     5-24-1988
38     7/27/1986
39     1-14-1981
40     7-29-1975
41     6-24-1987
42     8-14-1994
43     4/13/2002
44     8-16-1982
45     2/15/1998
46     7-15-1991
47    06-12-1994
48     9-17-1984
49     2-28-1975
Name: 0, dtype: object

In [270]:
len(date_sorter().dropna())

500

In [195]:
date_sorter().head(50)

C:\Users\cjr21\AppData\Local\Temp\ipykernel_3564\4241603988.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates["Date"] = pd.to_datetime(dates[0], errors="coerce", yearfirst=True)


,0,Date
0,03/25/93,1993-03-25
1,6/18/85,1985-06-18
2,7/8/71,2071-07-08
3,9/27/75,1975-09-27
4,2/6/96,1996-02-06
5,7/06/79,1979-07-06
6,5/18/78,1978-05-18
7,10/24/89,1989-10-24
8,3/7/86,1986-03-07
9,4/10/71,2071-04-10


In [165]:
df.iloc[0]

'03/25/93 Total time of visit (in minutes):\n'